In [43]:
import selenium
import json
import time
import re
import string
import requests
import bs4
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait


In [44]:
#This is the English version of the booking domain
domain =  'https://www.booking.com/index.html?lang=en-us'


In [45]:
def prepare_driver(url):
    '''Returns a Firefox Webdriver.'''
    options = Options()
    #options.add_argument('-headless')
    driver = Firefox(executable_path='geckodriver', options=options)
    driver.get(url)
    wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'ss')))
    return driver


In [46]:
def fill_form(driver, search_argument):
    '''Finds all the input tags in form and makes a POST requests.'''
    search_field = driver.find_element_by_id('ss')
    search_field.send_keys(search_argument)
    # We look for the search button and click it
    driver.find_element_by_class_name('sb-searchbox__button').click()
    wait = WebDriverWait(driver, timeout=10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'sr-hotel__title')))

In [47]:
def scrape_results(driver, n_results):
    '''Returns the data from n_results amount of results.'''

    accommodations_urls = list()
    accommodations_data = list()

    for accomodation_title in driver.find_elements_by_class_name('sr-hotel__title'):
        accommodations_urls.append(accomodation_title.find_element_by_class_name('hotel_name_link').get_attribute('href'))

    for url in range(0, n_results):
        if url == n_results:
            break
        url_data = scrape_accommodation_data(driver, accommodations_urls[url])
        accommodations_data.append(url_data)

    return accommodations_data

In [48]:
def scrape_accommodation_data(driver, accommodation_url):
    '''Visits an accommodation page and extracts the data.'''

    if driver == None:
        driver = prepare_driver(accommodation_url)

    driver.get(accommodation_url)
    time.sleep(5)

    accommodation_fields = dict()

    # Get the Hotel name to print
    hotel_name = driver.find_element_by_id('hp_hotel_name').text
    print(hotel_name + "\n=====================================\n")

    #Show the reviews of the hotel
    driver.find_element_by_id('show_reviews_tab').click()
    time.sleep(5)

    # find the number of pages with the reviews results
    rev_pages = int (driver.find_element_by_xpath("/html/body/div[5]/div/div[3]/div[2]/div[1]/div[11]/div/div[1]/div[2]/div[2]/div[3]/div[3]/div[4]/div/div[1]/div/div[2]/div/div[7]/a/span[1]").text)
    print ("The total number of pages with reviews is: " + str(rev_pages) + "\n\n")

    #First Page of reviews
    print("FIRST PAGE-------------------------------------\n")
    reviews = driver.find_elements_by_class_name('c-review__body') #this is the first page
    ii=0
    for rev in reviews:
        print (str(ii+1) + ". " + rev.text)
        ii = ii+1

    # Second page of reviews
    for ind in range (2,rev_pages+1):
        rev_link = driver.find_element_by_class_name('pagenext').get_attribute('href')
        driver.get(rev_link)
        time.sleep(5)
        print("PAGE " + str(ind) + " -------------------------------------\n")
        reviews = driver.find_elements_by_class_name('c-review__body')
        ii=0
        for rev in reviews:
            print (str(ii+1) + ". " + rev.text)
            ii=ii+1



    '''
    # Get the accommodation name
    accommodation_fields['name'] = driver.find_element_by_id('hp_hotel_name').text.strip('Hotel')

    # Get the accommodation score
    accommodation_fields['score'] = driver.find_element_by_class_name('bui-review-score--end').find_element_by_class_name('bui-review-score__badge').text

    # Get the accommodation location
    accommodation_fields['location'] = driver.find_element_by_id('showMap2').find_element_by_class_name('hp_address_subtitle').text

    # Get the most popular facilities
    accommodation_fields['popular_facilities'] = list()
    facilities = driver.find_element_by_class_name('hp_desc_important_facilities')

    for facility in facilities.find_elements_by_class_name('important_facility'):
        accommodation_fields['popular_facilities'].append(facility.text)
    '''
    return accommodation_fields



In [ ]:
if __name__ == '__main__':

    try:
        driver = prepare_driver(domain)
        fill_form(driver, 'Delphi')
        accommodations_data = scrape_results(driver, 1)
        accommodations_data = json.dumps(accommodations_data, indent=4)
        with open('booking_data.json', 'w') as f:
            f.write(accommodations_data)
    finally:
        driver.quit()
